<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/16AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

The dummy_input is a PyTorch tensor created with torch.rand(4, 3, 224, 224). This means it's a tensor of random numbers with the following dimensions:

4: Represents a batch size of 4. In machine learning, data is often processed in batches.
3: This typically refers to the number of color channels (e.g., Red, Green, Blue) for an image.
224, 224: These are the height and width of the 'dummy images', respectively. AlexNet expects input images of this size.

The dummy_labels is a PyTorch tensor created with torch.randint(0, 10, (4,)). This means it's a tensor of random integers with the following characteristics:

torch.randint(0, 10, ...): Generates integers randomly chosen from the range of 0 (inclusive) to 10 (exclusive), meaning the labels will be integers between 0 and 9.
(4,): This is the shape of the tensor, indicating it is a 1-dimensional tensor containing 4 elements. Each element corresponds to a label for one of the 4 dummy input images.

In [2]:
dummy_input = torch.rand(4, 3, 224, 224)
dummy_labels = torch.randint(0, 10, (4,))

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.LazyConv2d(96, kernel_size=11, stride=4, padding=1),
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LazyConv2d(256, kernel_size=5, padding=2), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LazyConv2d(384, kernel_size=3, padding=1), nn.ReLU(),
            nn.LazyConv2d(384, kernel_size=3, padding=1), nn.ReLU(),
            nn.LazyConv2d(256, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2), nn.Flatten(),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(p=0.5),
            nn.LazyLinear(4096), nn.ReLU(),nn.Dropout(p=0.5),
            nn.LazyLinear(num_classes))

    def forward(self, x):
        return self.net(x)

In [4]:
model = AlexNet()

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
model.train()
num_epochs = 5

for epoch in range(num_epochs):
    # Zero gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(dummy_input)

    # Calculate loss
    loss = criterion(outputs, dummy_labels)

    # Backpropagation
    loss.backward()

    # Update weights
    optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

Epoch 1/5, Loss: 2.3095
Epoch 2/5, Loss: 1.6188
Epoch 3/5, Loss: 2.0351
Epoch 4/5, Loss: 1.2224
Epoch 5/5, Loss: 3.9895
